In [2]:
from dotenv import load_dotenv
import os
from pyairtable import Api
from pyairtable.formulas import match
from datetime import datetime
from models import *
import pandas as pd
from pathlib import Path


load_dotenv()
KEY = os.getenv('KEY')
app = "app7KsgYl2jhOnYg7"

api = Api(KEY)
base = api.base(app)

home_dir = Path("../")
data_dir = home_dir / "data"
raw_dir = home_dir / "raw"

In [ ]:
existing_samples = [{"id": sample.id, "sample_id": sample.sample_id} for sample in Sample.all()]
df = pd.read_csv("meta/tissue_meta.csv")

In [ ]:
existing_samples = pd.DataFrame(existing_samples)
df['sample_id'] = df['sample_id'].astype(str)

In [ ]:
new_samples = pd.merge(df, existing_samples, "left", on="sample_id")
new_samples = new_samples[new_samples['id'].isna()]

In [ ]:
samples = []
for _, row in new_samples.itterrows():
    sample_id = row.get("sample_id")

    animal_id = row.get("animal")
    animal = [Animal.first(animal_id = animal_id)]

    sample_type = row.get("sample_type")
    mortem = row.get("mortem")
    sample_type_id = [
        SampleType.first(
            sample_type = sample_type,
            mortem = mortem
        )
    ]

    concentration = row.get("concentration")
    bilateral = row.get("bilateral")
    process_date = row.get("process_date")

    sample = Sample(
        sample_id = sample_id,
        animal = animal,
        sample_type_id = sample_type_id,
        concentration = concentration,
        bilateral = bilateral
    )

    samples.append(sample)